In [2]:
import gensim
from gensim.models import Word2Vec , KeyedVectors

In [3]:
import gensim.downloader as api
wv = api.load('word2vec-google-news-300')
vec_king  = wv['king']

In [5]:
import pandas as pd
messages = pd.read_csv("sms_spam.csv",names = ["label","message"])
messages = messages.drop(0)
messages = messages.reset_index(drop=True)


In [6]:
messages.head()

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [7]:
vec_king.shape

(300,)

In [8]:
from nltk.stem import WordNetLemmatizer

In [9]:
lemmitizer = WordNetLemmatizer()

In [10]:
# Data Cleaning and preprocessing
import re 
import nltk

In [11]:
corpus = []

for i in range(len(messages)):
    review = re.sub('[^a-zA-Z]',' ', messages['message'][i])
    review = review.lower()
    review = review.split()
    review = [lemmitizer.lemmatize(word) for word in review]
    review = ' '.join(review)
    corpus.append(review)

In [57]:
[[i,j,k] for i,j,k in zip(list(map(len,corpus)),corpus, messages['message']) if i<1]

[[0, '', '645'], [0, '', ':)'], [0, '', ':-) :-)']]

In [34]:
# corpus

In [13]:
from nltk import sent_tokenize
from gensim.utils import simple_preprocess

In [14]:
words = []
for sent in corpus:
    sent_token = sent_tokenize(sent)
    for sent in sent_token:
        words.append(simple_preprocess(sent))

In [16]:
# words

In [17]:
#  Lets train Word2vec fro mscratch 
model = gensim.models.Word2Vec(words,vector_size=100)

In [33]:
# To Get all the vocubulary
# model.wv.index_to_key

In [19]:
model.corpus_count

5571

In [21]:
model.epochs

5

In [23]:
model.wv.similar_by_word('good')

[('my', 0.9989002346992493),
 ('wa', 0.9986514449119568),
 ('all', 0.9985989332199097),
 ('morning', 0.9985649585723877),
 ('day', 0.9985515475273132),
 ('amp', 0.9984886050224304),
 ('very', 0.9984011054039001),
 ('great', 0.9983295202255249),
 ('where', 0.9983265399932861),
 ('of', 0.9983262419700623)]

In [25]:
model.wv['good'].shape

(100,)

In [26]:
words[0]

['go',
 'until',
 'jurong',
 'point',
 'crazy',
 'available',
 'only',
 'in',
 'bugis',
 'great',
 'world',
 'la',
 'buffet',
 'cine',
 'there',
 'got',
 'amore',
 'wat']

In [42]:
def avg_word2vec(doc):
    return np.mean([model.wv[word] for word in doc if word in model.wv.index_to_key],axis = 0)

In [43]:
from tqdm import tqdm
import numpy as np

In [44]:
# apply for the entire sentence 
X = []
for i in tqdm(range(len(words))):
    # print("Hello",i)
    X.append(avg_word2vec(words[i]))

100%|████████████████████████████████████████████████████████████████████████████| 5571/5571 [00:01<00:00, 4025.99it/s]


In [45]:
X[0]

array([-0.19314583,  0.2506586 ,  0.04965586,  0.10677772,  0.07574973,
       -0.5180272 ,  0.13750014,  0.50186056, -0.2505655 , -0.09551864,
       -0.20511647, -0.44642544, -0.05420703,  0.05336574,  0.19486675,
       -0.15658149,  0.12472589, -0.31246465, -0.07017021, -0.54471093,
        0.17954502,  0.23512734,  0.10307756, -0.1646872 , -0.12281334,
       -0.01257183, -0.36929706, -0.21066888, -0.26204917,  0.00353965,
        0.29352838, -0.00453415,  0.07984068, -0.18713266, -0.18262018,
        0.40921515,  0.03908635, -0.20282452, -0.13022535, -0.3833567 ,
        0.1857749 , -0.26368073, -0.22786628,  0.03387659,  0.261595  ,
       -0.00055831, -0.19543079, -0.0930147 ,  0.11809704,  0.17867573,
        0.14977258, -0.17191419, -0.06031509,  0.08176192, -0.12850687,
        0.18606846,  0.20166592,  0.08347165, -0.30455923,  0.113457  ,
       -0.04655141,  0.1510706 , -0.06329656, -0.14524981, -0.30138832,
        0.24559344,  0.0525254 ,  0.1470534 , -0.34481   ,  0.28

In [46]:
X[0].shape

(100,)

In [48]:
X_new = np.array(X, dtype=object)

In [51]:
X_new[0].shape

(100,)

In [58]:
# Dependent Features
y = messages[list(map(lambda x:len(x)>0 , corpus))]
y = pd.get_dummies(y['label'])
y = y.astype(int)
y = y.iloc[:,0].values

In [59]:
y.shape

(5571,)

In [56]:
X_new.shape

(5571,)

In [62]:
# thiss is the finla independent features
df = pd.DataFrame()
for i in range(len(X)):
    row = pd.DataFrame(X[i].reshape(1, -1))
    df = pd.concat([df, row], ignore_index=True)

C:\Users\hp\AppData\Local\Temp\ipykernel_25104\1882536028.py:5: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, row], ignore_index=True)


In [63]:
df.shape

(5571, 100)

In [64]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-0.193146,0.250659,0.049656,0.106778,0.075750,-0.518027,0.137500,0.501861,-0.250565,-0.095519,...,0.252947,0.148368,0.090835,0.138706,0.355235,0.348872,0.145552,-0.194594,0.171911,-0.094510
1,-0.169615,0.220215,0.038091,0.096771,0.059187,-0.464110,0.114240,0.448009,-0.214370,-0.070630,...,0.224438,0.127319,0.079112,0.120354,0.309823,0.300062,0.127983,-0.177526,0.144608,-0.082801
2,-0.188458,0.263186,0.076692,0.136669,0.091017,-0.569461,0.139970,0.511552,-0.266072,-0.107475,...,0.247991,0.158480,0.078570,0.149244,0.370861,0.348915,0.127930,-0.228488,0.202313,-0.100720
3,-0.266529,0.345688,0.060758,0.148251,0.093277,-0.701900,0.186215,0.692005,-0.345532,-0.120741,...,0.347486,0.197404,0.121223,0.190278,0.473388,0.478040,0.200533,-0.270248,0.225504,-0.119531
4,-0.229615,0.287335,0.055982,0.119452,0.096437,-0.586653,0.149972,0.584148,-0.290189,-0.107833,...,0.302134,0.161976,0.103132,0.157136,0.397084,0.402446,0.161530,-0.233059,0.191218,-0.109472


In [72]:
df['Output'] = y

In [73]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,Output
0,-0.193146,0.250659,0.049656,0.106778,0.075750,-0.518027,0.137500,0.501861,-0.250565,-0.095519,...,0.148368,0.090835,0.138706,0.355235,0.348872,0.145552,-0.194594,0.171911,-0.094510,1
1,-0.169615,0.220215,0.038091,0.096771,0.059187,-0.464110,0.114240,0.448009,-0.214370,-0.070630,...,0.127319,0.079112,0.120354,0.309823,0.300062,0.127983,-0.177526,0.144608,-0.082801,1
2,-0.188458,0.263186,0.076692,0.136669,0.091017,-0.569461,0.139970,0.511552,-0.266072,-0.107475,...,0.158480,0.078570,0.149244,0.370861,0.348915,0.127930,-0.228488,0.202313,-0.100720,0
3,-0.266529,0.345688,0.060758,0.148251,0.093277,-0.701900,0.186215,0.692005,-0.345532,-0.120741,...,0.197404,0.121223,0.190278,0.473388,0.478040,0.200533,-0.270248,0.225504,-0.119531,1
4,-0.229615,0.287335,0.055982,0.119452,0.096437,-0.586653,0.149972,0.584148,-0.290189,-0.107833,...,0.161976,0.103132,0.157136,0.397084,0.402446,0.161530,-0.233059,0.191218,-0.109472,1


In [74]:
df.dropna(inplace = True)

In [75]:
df.isnull().sum()

0         0
1         0
2         0
3         0
4         0
         ..
96        0
97        0
98        0
99        0
Output    0
Length: 101, dtype: int64

In [81]:
# Independent Featrues
X = df.iloc[:,:-1]
y = df['Output']

In [82]:
X.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-0.193146,0.250659,0.049656,0.106778,0.075750,-0.518027,0.137500,0.501861,-0.250565,-0.095519,...,0.252947,0.148368,0.090835,0.138706,0.355235,0.348872,0.145552,-0.194594,0.171911,-0.094510
1,-0.169615,0.220215,0.038091,0.096771,0.059187,-0.464110,0.114240,0.448009,-0.214370,-0.070630,...,0.224438,0.127319,0.079112,0.120354,0.309823,0.300062,0.127983,-0.177526,0.144608,-0.082801
2,-0.188458,0.263186,0.076692,0.136669,0.091017,-0.569461,0.139970,0.511552,-0.266072,-0.107475,...,0.247991,0.158480,0.078570,0.149244,0.370861,0.348915,0.127930,-0.228488,0.202313,-0.100720
3,-0.266529,0.345688,0.060758,0.148251,0.093277,-0.701900,0.186215,0.692005,-0.345532,-0.120741,...,0.347486,0.197404,0.121223,0.190278,0.473388,0.478040,0.200533,-0.270248,0.225504,-0.119531
4,-0.229615,0.287335,0.055982,0.119452,0.096437,-0.586653,0.149972,0.584148,-0.290189,-0.107833,...,0.302134,0.161976,0.103132,0.157136,0.397084,0.402446,0.161530,-0.233059,0.191218,-0.109472


In [83]:
# Train test split
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.2,random_state=42)

In [84]:
X_train

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
1394,-0.253892,0.325083,0.051735,0.132521,0.098643,-0.676376,0.181469,0.668956,-0.319082,-0.110140,...,0.337558,0.186869,0.121505,0.185745,0.459522,0.456049,0.198371,-0.252113,0.213100,-0.121329
2637,-0.206847,0.269883,0.055559,0.106884,0.093296,-0.542777,0.141359,0.533663,-0.260072,-0.105587,...,0.275492,0.152650,0.097975,0.140907,0.369959,0.368935,0.152270,-0.207532,0.189255,-0.098776
2578,-0.214293,0.280034,0.053804,0.108926,0.084683,-0.554044,0.147479,0.546032,-0.272045,-0.097005,...,0.269986,0.169878,0.115131,0.148880,0.389347,0.391724,0.171398,-0.200469,0.199431,-0.111680
1257,-0.200907,0.259946,0.055210,0.113288,0.078292,-0.525014,0.135694,0.514949,-0.252450,-0.091454,...,0.261504,0.148270,0.092163,0.137691,0.358282,0.359163,0.143001,-0.197474,0.177604,-0.096454
4235,-0.230583,0.309778,0.088071,0.153983,0.106375,-0.664703,0.165398,0.603438,-0.311981,-0.121303,...,0.290672,0.187489,0.098111,0.175895,0.438097,0.415390,0.160646,-0.259856,0.233971,-0.111767
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3778,-0.226008,0.305758,0.097704,0.153123,0.121948,-0.658285,0.166960,0.590415,-0.308337,-0.145389,...,0.304418,0.172313,0.074385,0.150736,0.430900,0.387247,0.126639,-0.280906,0.233443,-0.100490
5202,-0.227699,0.268384,0.055290,0.112056,0.108915,-0.550001,0.135673,0.544327,-0.273081,-0.112173,...,0.302164,0.151803,0.091954,0.144388,0.364143,0.373598,0.143290,-0.227102,0.183911,-0.097294
5237,-0.250289,0.285364,0.045115,0.111112,0.112594,-0.588153,0.144879,0.599114,-0.294313,-0.112911,...,0.332574,0.157103,0.107973,0.143402,0.389968,0.409985,0.156061,-0.238238,0.194833,-0.103235
5401,-0.254676,0.310054,0.058149,0.128065,0.107877,-0.634840,0.160752,0.629592,-0.313141,-0.122749,...,0.332057,0.175892,0.112229,0.164583,0.425198,0.437076,0.175027,-0.250209,0.214550,-0.114956


In [85]:
from sklearn.ensemble import RandomForestClassifier
classifer = RandomForestClassifier()

In [86]:
classifer.fit(X_train,y_train)

,n_estimators,100
,criterion,'gini'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [87]:
y_pred = classifer.predict(X_test)

In [88]:
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix

In [90]:
print(accuracy_score(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))

0.9685251798561151
              precision    recall  f1-score   support

           0       0.93      0.82      0.87       142
           1       0.97      0.99      0.98       970

    accuracy                           0.97      1112
   macro avg       0.95      0.90      0.93      1112
weighted avg       0.97      0.97      0.97      1112

[[116  26]
 [  9 961]]
